# **Model Version: 0.1 + gridsearch**


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### ***Imports***

In [38]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import os
import numpy as np
import librosa
import random
from torch.utils.data import DataLoader
from torch import optim
import argparse
from sklearn.metrics import accuracy_score

### **Models**

In [39]:
class TDNN(nn.Module):
    
    def __init__(
                    self, 
                    input_dim=23, 
                    output_dim=512,
                    context_size=5,
                    stride=1,
                    dilation=1,
                    batch_norm=False,
                    dropout_p=0.2
                ):
        '''
        TDNN as defined by https://www.danielpovey.com/files/2015_interspeech_multisplice.pdf

        Affine transformation not applied globally to all frames but smaller windows with local context

        batch_norm: True to include batch normalisation after the non linearity
        
        Context size and dilation determine the frames selected
        (although context size is not really defined in the traditional sense)
        For example:
            context size 5 and dilation 1 is equivalent to [-2,-1,0,1,2]
            context size 3 and dilation 2 is equivalent to [-2, 0, 2]
            context size 1 and dilation 1 is equivalent to [0]
        '''
        super(TDNN, self).__init__()
        self.context_size = context_size
        self.stride = stride
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.dilation = dilation
        self.dropout_p = dropout_p
        self.batch_norm = batch_norm
      
        self.kernel = nn.Linear(input_dim*context_size, output_dim)
        self.nonlinearity = nn.ReLU()
        if self.batch_norm:
            self.bn = nn.BatchNorm1d(output_dim)
        if self.dropout_p:
            self.drop = nn.Dropout(p=self.dropout_p)
        
    def forward(self, x):
        '''
        input: size (batch, seq_len, input_features)
        outpu: size (batch, new_seq_len, output_features)
        '''
        
        _, _, d = x.shape
        assert (d == self.input_dim), 'Input dimension was wrong. Expected ({}), got ({})'.format(self.input_dim, d)
        x = x.unsqueeze(1)

        # Unfold input into smaller temporal contexts
        x = F.unfold(
                        x, 
                        (self.context_size, self.input_dim), 
                        stride=(1,self.input_dim), 
                        dilation=(self.dilation,1)
                    )

        # N, output_dim*context_size, new_t = x.shape
        x = x.transpose(1,2)
        x = self.kernel(x.float())
        x = self.nonlinearity(x)
        
        if self.dropout_p:
            x = self.drop(x)

        if self.batch_norm:
            x = x.transpose(1,2)
            x = self.bn(x)
            x = x.transpose(1,2)

        return x

In [40]:
class X_vector(nn.Module):
    # changed for grid-search starts
    def __init__(self, input_dim = 40, num_classes=8, context_size_layer_1=5, dilation_layer_1=1, context_size_layer_2=3, dilation_layer_2=1, context_size_layer_3=2, dilation_layer_3=2):
        super(X_vector, self).__init__()
        i_d = input_dim
        self.tdnn1 = TDNN(input_dim=i_d, output_dim=512, context_size=context_size_layer_1, dilation=dilation_layer_1, dropout_p=0.5)
        self.tdnn2 = TDNN(input_dim=512, output_dim=512, context_size=context_size_layer_2, dilation=dilation_layer_2, dropout_p=0.5)
        self.tdnn3 = TDNN(input_dim=512, output_dim=512, context_size=context_size_layer_3, dilation=dilation_layer_3, dropout_p=0.5)
    # changed for grid-search end
        self.tdnn4 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=1,dropout_p=0.5)
        self.tdnn5 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=3,dropout_p=0.5)
        #### Frame levelPooling
        self.segment6 = nn.Linear(1024, 512)
        self.segment7 = nn.Linear(512, 512)
        self.output = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, inputs):
        tdnn1_out = self.tdnn1(inputs)
        return tdnn1_out
        tdnn2_out = self.tdnn2(tdnn1_out)
        tdnn3_out = self.tdnn3(tdnn2_out)
        tdnn4_out = self.tdnn4(tdnn3_out)
        tdnn5_out = self.tdnn5(tdnn4_out)
        ### Stat Pool
        mean = torch.mean(tdnn5_out,1)
        std = torch.std(tdnn5_out,1)
        stat_pooling = torch.cat((mean,std),1)
        segment6_out = self.segment6(stat_pooling)
        x_vec = self.segment7(segment6_out)
        predictions = self.softmax(self.output(x_vec))
        return predictions,x_vec

In [41]:
class X_vector(nn.Module):
  # changed for grid-search starts
    def __init__(self, input_dim = 40, num_classes=8, context_size_layer_1=5, dilation_layer_1=1, context_size_layer_2=3, dilation_layer_2=1, context_size_layer_3=2, dilation_layer_3=2):
        super(X_vector, self).__init__()
        i_d = input_dim
        self.tdnn1 = TDNN(input_dim=i_d, output_dim=512, context_size=context_size_layer_1, dilation=dilation_layer_1, dropout_p=0.5)
        self.tdnn2 = TDNN(input_dim=512, output_dim=512, context_size=context_size_layer_2, dilation=dilation_layer_2, dropout_p=0.5)
        self.tdnn3 = TDNN(input_dim=512, output_dim=512, context_size=context_size_layer_3, dilation=dilation_layer_3, dropout_p=0.5)
  # changed for grid-search end
        self.tdnn4 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=1, dropout_p=0.5)
        self.tdnn5 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=3, dropout_p=0.5)
        #### Frame levelPooling
        self.segment6 = nn.Linear(1024, 512)
        self.segment7 = nn.Linear(512, 512)
        self.output = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, inputs):
        tdnn1_out = self.tdnn1(inputs)
        tdnn2_out = self.tdnn2(tdnn1_out)
        tdnn3_out = self.tdnn3(tdnn2_out)
        tdnn4_out = self.tdnn4(tdnn3_out)
        tdnn5_out = self.tdnn5(tdnn4_out)
        ### Stat Pool
        
        mean = torch.mean(tdnn5_out,1)
        std = torch.var(tdnn5_out,1)
        stat_pooling = torch.cat((mean,std),1)
        segment6_out = self.segment6(stat_pooling)
        x_vec = self.segment7(segment6_out)
        predictions = self.output(x_vec)
        return predictions,x_vec

### Utiles 

In [42]:
def load_wav(audio_filepath, sr, min_dur_sec=4):
    audio_data,fs  = librosa.load(audio_filepath,sr=16000)
    len_file = len(audio_data)
    
    if len_file <int(min_dur_sec*sr):
        dummy=np.zeros((1,int(min_dur_sec*sr)-len_file))
        extened_wav = np.concatenate((audio_data,dummy[0]))
    else:
        
        extened_wav = audio_data
    return extened_wav


def lin_mel_from_wav(wav, hop_length, win_length, n_mels):
    linear = librosa.feature.melspectrogram(wav, n_mels=n_mels, win_length=win_length, hop_length=hop_length) # linear spectrogram
    return linear.T

def lin_spectogram_from_wav(wav, hop_length, win_length, n_fft=512):
    linear = librosa.stft(wav, n_fft=n_fft, win_length=win_length, hop_length=hop_length) # linear spectrogram
    return linear.T


def feature_extraction(filepath,sr=16000, min_dur_sec=4,win_length=400,hop_length=160, n_mels=40, spec_len=400,mode='train'):
    audio_data = load_wav(filepath, sr=sr,min_dur_sec=min_dur_sec)
    linear_spect = lin_spectogram_from_wav(audio_data, hop_length, win_length, n_fft=512)
    mag, _ = librosa.magphase(linear_spect)  # magnitude
    mag_T = mag.T
    mu = np.mean(mag_T, 0, keepdims=True)
    std = np.std(mag_T, 0, keepdims=True)
    return (mag_T - mu) / (std + 1e-5)
    
    
    
    
def load_data(filepath,sr=16000, min_dur_sec=4,win_length=400,hop_length=160, n_mels=40, spec_len=400,mode='train'):
    audio_data = load_wav(filepath, sr=sr,min_dur_sec=min_dur_sec)
    #linear_spect = lin_spectogram_from_wav(audio_data, hop_length, win_length, n_mels)
    linear_spect = lin_spectogram_from_wav(audio_data, hop_length, win_length, n_fft=512)
    mag, _ = librosa.magphase(linear_spect)  # magnitude
    mag_T = mag.T
    
    if mode=='train':
        randtime = np.random.randint(0, mag_T.shape[1]-spec_len)
        spec_mag = mag_T[:, randtime:randtime+spec_len]
    else:
        spec_mag = mag_T
    
    # preprocessing, subtract mean, divided by time-wise var
    mu = np.mean(spec_mag, 0, keepdims=True)
    std = np.std(spec_mag, 0, keepdims=True)
    return (spec_mag - mu) / (std + 1e-5)
    


def load_npy_data(filepath,spec_len=400,mode='train'):
    mag_T = np.load(filepath)
    if mode=='train':
        randtime = np.random.randint(0, mag_T.shape[1]-spec_len)
        spec_mag = mag_T[:, randtime:randtime+spec_len]
    else:
        spec_mag = mag_T
    return spec_mag
    
    



def speech_collate(batch):
    targets = []
    specs = []
    for sample in batch:
        specs.append(sample['features'])
        targets.append((sample['labels']))
    return specs, targets

## SpeechDataGenerator

In [43]:
class SpeechDataGenerator():
    """Speech dataset."""

    def __init__(self, manifest, mode):
        """
        Read the textfile and get the paths
        """
        
        # [line.rstrip('\n').split(' ')[0]
        self.mode=mode
        self.audio_links = [" ".join(line.rstrip('\n').split(' ')[:-1]) for line in open(manifest)]
        self.labels = [int(line.rstrip('\n').split(' ')[-1]) for line in open(manifest)]
        

    def __len__(self):
        return len(self.audio_links)

    def __getitem__(self, idx):
        audio_link =self.audio_links[idx]
        class_id = self.labels[idx]
        #lang_label=lang_id[self.audio_links[idx].split('/')[-2]]
        spec = load_data(audio_link,mode=self.mode)
        sample = {'features': torch.from_numpy(np.ascontiguousarray(spec)), 'labels': torch.from_numpy(np.ascontiguousarray(class_id))}
        return sample

## Train X Vector

In [46]:
torch.multiprocessing.set_sharing_strategy('file_system')

training_filepath = '/content/drive/My Drive/10_lang_wav_files/train_10_lang.txt'
testing_filepath = '/content/drive/My Drive/10_lang_wav_files/test_10_lang.txt'
validation_filepath = '/content/drive/My Drive/10_lang_wav_files/validation_10_lang.txt'
input_dim = 257
num_classes = 10
lamda_val = 0.5
batch_size = 10
use_gpu = True
num_epochs = 100

### Data related
dataset_train = SpeechDataGenerator(manifest=training_filepath, mode='train')
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

dataset_val = SpeechDataGenerator(manifest=validation_filepath, mode='train')
dataloader_val = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

dataset_test = SpeechDataGenerator(manifest=testing_filepath, mode='test')
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

## Model related
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = X_vector(input_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
loss_fun = nn.CrossEntropyLoss()


def train(dataloader_train, epoch):
    train_loss_list = []
    full_preds = []
    full_gts = []
    model.train()

    for i_batch, sample_batched in enumerate(dataloader_train):
        print("batch: " + str(i_batch+1))
        features = torch.from_numpy(np.asarray([torch_tensor.numpy().T for torch_tensor in sample_batched[0]])).float()
        labels = torch.from_numpy(np.asarray([torch_tensor[0].numpy() for torch_tensor in sample_batched[1]]))
        labels = labels.long()
        features, labels = features.to(device), labels.to(device)
        features.requires_grad = True
        optimizer.zero_grad()
        pred_logits, x_vec = model(features)
        #### CE loss
        loss = loss_fun(pred_logits, labels)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        # train_acc_list.append(accuracy)
        # if i_batch%10==0:
        #    print('Loss {} after {} iteration'.format(np.mean(np.asarray(train_loss_list)),i_batch))

        predictions = np.argmax(pred_logits.detach().cpu().numpy(), axis=1)
        for pred in predictions:
            full_preds.append(pred)
        for lab in labels.detach().cpu().numpy():
            full_gts.append(lab)

    mean_acc = accuracy_score(full_gts, full_preds)
    mean_loss = np.mean(np.asarray(train_loss_list))
    print('Total training loss {} and training Accuracy {} after {} epochs'.format(mean_loss, mean_acc, epoch))


def validation(dataloader_val, epoch):
    model.eval()
    with torch.no_grad():
        val_loss_list = []
        full_preds = []
        full_gts = []
        for i_batch, sample_batched in enumerate(dataloader_val):
            features = torch.from_numpy(
                np.asarray([torch_tensor.numpy().T for torch_tensor in sample_batched[0]])).float()
            labels = torch.from_numpy(np.asarray([torch_tensor[0].numpy() for torch_tensor in sample_batched[1]]))
            labels = labels.long()
            features, labels = features.to(device), labels.to(device)
            pred_logits, x_vec = model(features)
            #### CE loss
            loss = loss_fun(pred_logits, labels)
            val_loss_list.append(loss.item())
            # train_acc_list.append(accuracy)
            predictions = np.argmax(pred_logits.detach().cpu().numpy(), axis=1)
            for pred in predictions:
                full_preds.append(pred)
            for lab in labels.detach().cpu().numpy():
                full_gts.append(lab)

        mean_acc = accuracy_score(full_gts, full_preds)
        mean_loss = np.mean(np.asarray(val_loss_list))
        print('Total validation loss {} and Validation accuracy {} after {} epochs'.format(mean_loss, mean_acc, epoch))

        # TODO: CHANGE BACK TO SAVE MODEL:
        # model_save_path = os.path.join('save_model', 'best_check_point_' + str(epoch) + '_' + str(mean_loss))
        # state_dict = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch}
        # torch.save(state_dict, model_save_path)

RuntimeError: ignored

In [35]:
# ### Data related
# dataset_train = SpeechDataGenerator(manifest=training_filepath, mode='train')
# dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

# dataset_val = SpeechDataGenerator(manifest=validation_filepath, mode='train')
# dataloader_val = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

# dataset_test = SpeechDataGenerator(manifest=testing_filepath, mode='test')
# dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

# ## Model related
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
# # model = X_vector(input_dim, num_classes).to(device)
# # optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
# # loss_fun = nn.CrossEntropyLoss()

In [36]:
#gridSearch start:

import itertools
num_epochs = 5

context_sizes = [2, 3, 5, 7]
dilations = [1, 2, 3]

best_accuracy = 0.0
best_hyperparams = None
hyperparameters_list = []
accuracy_and_loss_list = []

for context_size, dilation in itertools.product(context_sizes, dilations):
    print()
    print(f'Training with context_size={context_size} and dilation={dilation}')
    print()

    # Initialize the model with the current hyperparameters for layer 1 (only!)
    # TODO: change to 3 layers in some way
    model = X_vector(input_dim, num_classes, context_size_layer_1=context_size, dilation_layer_1=dilation).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
    loss_fun = nn.CrossEntropyLoss()

    # Train and validate the model
    for epoch in range(num_epochs):
        print("epoch: " + str(epoch))
        train(dataloader_train, epoch)
        accuracy_and_loss = validation(dataloader_val, epoch)
        

        if accuracy_and_loss[0] > best_accuracy:
            best_accuracy = accuracy_and_loss[0]
            best_hyperparams = (context_size, dilation)
    
    print(f'hyperparameters (context_size,dilation): ={hyperparameters_list}, accuracy={accuracy_and_loss_list}')
    hyperparameters_list.append((context_size, dilation))
    accuracy_and_loss_list.append(accuracy_and_loss)


print(f'Best hyperparameters: context_size={best_hyperparams[0]}, dilation={best_hyperparams[1]}, '
      f'validation accuracy={best_accuracy}')

print(f'hyperparameters (context_size,dilation): ={hyperparameters_list}, accuracy={accuracy_and_loss_list}')

#gridSearch end


Training with context_size=2 and dilation=1



RuntimeError: ignored

In [ ]:
# import itertools
# import concurrent.futures
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# num_epochs = 100
# context_sizes = [2, 3, 5, 7]
# dilations = [1, 2, 3]

# best_accuracy = 0.0
# best_hyperparams = None
# hyperparameters_list = []
# accuracy_and_loss_list = []

# def train_and_validate(context_size, dilation):
#     global best_accuracy
#     global best_hyperparams

#     print()
#     print(f'Training with context_size={context_size} and dilation={dilation}')
#     print()

#     # Initialize the model with the current hyperparameters for layer 1 (only!)
#     # TODO: change to 3 layers in some way
#     model = X_vector(input_dim, num_classes, context_size_layer_1=context_size, dilation_layer_1=dilation).to(device)
#     optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
#     loss_fun = nn.CrossEntropyLoss()

#     # Train and validate the model
#     for epoch in range(num_epochs):
#         # print("epoch: " + str(epoch))
#         train(model, optimizer, loss_fun, dataloader_train, epoch)
#         accuracy_and_loss = validation(model, loss_fun, dataloader_val, epoch)

#         if accuracy_and_loss[0] > best_accuracy:
#             best_accuracy = accuracy_and_loss[0]
#             best_hyperparams = (context_size, dilation)

#     return (context_size, dilation, accuracy_and_loss)

# # Create a thread pool with a maximum of 4 threads
# with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
#     futures = []
#     for context_size, dilation in itertools.product(context_sizes, dilations):
#         futures.append(executor.submit(train_and_validate, context_size, dilation))

#     for future in concurrent.futures.as_completed(futures):
#         result = future.result()
#         hyperparameters_list.append((result[0], result[1]))
#         accuracy_and_loss_list.append(result[2])
#         print(f'hyperparameters (context_size,dilation): {hyperparameters_list}, accuracy: {accuracy_and_loss_list}')

# print(f'Best hyperparameters: context_size={best_hyperparams[0]}, dilation={best_hyperparams[1]}, '
#       f'validation accuracy={best_accuracy}')

# print(f'hyperparameters (context_size,dilation): {hyperparameters_list}, accuracy: {accuracy_and_loss_list}')



Training with context_size=2 and dilation=1


Training with context_size=2 and dilation=2


Training with context_size=2 and dilation=3


Training with context_size=3 and dilation=1


Training with context_size=3 and dilation=2


Training with context_size=3 and dilation=3



Training with context_size=5 and dilation=2

Training with context_size=5 and dilation=1


Training with context_size=5 and dilation=3


Training with context_size=7 and dilation=1


Training with context_size=7 and dilation=2


Training with context_size=7 and dilation=3



RuntimeError: ignored